In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
path = '/Users/galina.ryazanskaya/Downloads/thesis?/2023_annotation/'

In [3]:
def read_eaf_tsv(file_path, col_name='Transcript', file_col_name='File') -> pd.DataFrame:
    data = pd.read_csv(file_path, sep='\t', low_memory=False)
    data[file_col_name] = data[file_col_name].apply(lambda x: x.replace('.eaf', '').replace('--', '-'))
    raw = data.groupby(file_col_name)[col_name].apply(list)
    return pd.DataFrame(raw.apply(preprocess_raw_transcript))
    

In [4]:
def replace_func(text):
    if type(text) != str:
        return ''
    text = text.replace('ё', 'е')
    text = text.replace('  ·  ', ' ')
    text = text.replace('\\', '/')
    text = re.sub('\[[а-я]+\]', '', text)
    text = re.sub('\(ехр:[ а-я.,?!]+\)', '', text) 
    text = re.sub('\(exp:[ а-я.,?!]+\)', '', text) 
    text = re.sub('\(exp:[ а-я.,?!]+\)', '', text) 
    text = re.sub('[а-я]*\([0-9]+(.[0-9]+)\)?', '', text)
    text = re.sub('#нрзб', '', text)
    text = re.sub('нрзб', '', text)
    text = re.sub('[0-9]+(.[0-9]+)?', '', text)
    text = re.sub('[а-я]+=|\*[а-я]+', '',  text)
    text = re.sub('аа+', '', text)
    text = re.sub('ээ+', '', text)
    text = re.sub('//', '// ', text)
    text = re.sub(' /', ' / ', text)
    text = re.sub('/ /', '//', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('    ', ' ', text)
    text = re.sub('   ', ' ', text)
    text = re.sub('  ', ' ', text)
    return text

In [5]:
def preprocess_raw_transcript(text_list):
    new = []
    for text in text_list:
        t = replace_func(text)
        if t:
            new.append(t)
    return ' '.join(new)

In [6]:
raw_transcript = read_eaf_tsv(path + 'outer_transcript.tsv')

In [7]:
raw_transcript_lex = read_eaf_tsv(path + 'outer_transcript_lex.tsv', 'Transcript_lex')

In [8]:
additional_transcript = read_eaf_tsv(path + 'additioanl_transcript.tsv')

In [9]:
additional_transcript_lex = read_eaf_tsv(path + 'additioanl_transcript_lex.tsv', 'Transcript_lex')
# does not have correct utt separataion

In [10]:
raw_transcript_lex_add = read_eaf_tsv(path + 'discourse_transcript_lex.tsv', 'Transcript_lex')

In [11]:
raw_transcript_lex_alt = read_eaf_tsv(path + 'altered_transcript_lex.tsv', 'Transcript_lex')

In [12]:
# this comes from 'all_transcripts.xcls' sent by maria

In [13]:
raw_transcript_lex_pt =read_eaf_tsv(path + 'patients.discourse.tsv', 'Transcript_lex', 'fileID')
raw_transcript_lex_pt.index.rename('File', inplace=True)

In [14]:
raw_transcript_lex_pt

,Transcript_lex
File,
N-01-instr-1-table,// необходимо на пол положить столешницу так ...
N-01-pers-1-present,// самым запоминающимся подарком был последни...
N-01-pic-1-sportsman,// человек был очень завистливый // он увиде...
N-02-instr-1-chair,// переверните крышку табурета тыльной сторон...
N-02-pers-1-present,// однажды / когда я еще училась в школе -/ к...
...,...
S-57-instr-2-chair,// это значит берем табуретку собираем запчас...
S-57-pers-1-present,// самый запомнившийся подарок в моей жизни э...
S-57-pers-2-trip,// а люблю путешествовать к морю / когда нача...


In [15]:
ids_to_drop = list(set([i for i in raw_transcript_lex_pt.index if i.startswith('S')]))  # surgery patients, not schizophrenia
raw_transcript_lex_pt = raw_transcript_lex_pt.drop(index=ids_to_drop)

In [16]:
len(raw_transcript_lex_pt)

258

In [17]:
set(raw_transcript.index).difference(set(raw_transcript_lex.index))

{'PN-202-pers-1-party', 'PN-210-pers-1-party', 'PN-213-pers-1-present'}

In [18]:
set(raw_transcript_lex.index).difference(set(raw_transcript.index))

{'PN-201-pers-1-party'}

In [19]:
df = pd.merge(raw_transcript_lex, raw_transcript, on="File", how="outer")
df = df.fillna('')
df['outer'] = df.apply(lambda a: a['Transcript'] if (('//' in a['Transcript'] and not '//' in a['Transcript_lex']) or len(a['Transcript']) > len(a['Transcript_lex'])) else a['Transcript_lex'], axis=1)

In [20]:
df = pd.merge(raw_transcript_lex_add, df['outer'], on="File", how="outer")
df = df.fillna('')
# df['add'] = df.apply(lambda a: a['Transcript_new'] if (('//' in a['Transcript_new'] and not '//' in a['Transcript_lex']) or len(a['Transcript_new']) > len(a['Transcript_lex'])) else a['Transcript_lex'], axis=1)
df = df.rename({'Transcript_lex':'discourse'}, axis='columns')

In [21]:
df = pd.merge(raw_transcript_lex_alt, df[['outer', 'discourse']], on="File", how="outer")
df = df.fillna('')
# df['alt'] = df.apply(lambda a: a['Transcript_new'] if (('//' in a['Transcript_new'] and not '//' in a['Transcript_lex']) or len(a['Transcript_new']) > len(a['Transcript_lex'])) else a['Transcript_lex'], axis=1)
df = df.rename({'Transcript_lex':'altered'}, axis='columns')
df

,altered,outer,discourse
File,,,
PD-004-instr-1-chair,// я не знаю / как называются некоторые детал...,,/ я не знаю / как называются некоторые детали ...
PD-006-pers-1-present,// который я дарила // или который мне дарил...,,/ который я дарила / или который мне дарили та...
PD-007-instr-1-chair,// мы собираем табуретку из икеи // берем сн...,,/ мы собираем табуретку из икеи / берем сначал...
PD-007-pic-1-sportsman,// угу // один мужчина встретил другого мужч...,,угу / один мужчина встретил другого мужчину / ...
PD-008-instr-1-chair,// так / ну взять сидушку стула первую часть ...,,/ так ну / взять сидушку стула а первую часть ...
...,...,...,...
PN-327-pic-1-sportsman,// жил один человек / который был очень худой...,/ жил один человек / который был очень худой и...,// жил один человек / который был очень худой...
PD-004-pers-1-present,,// когда я была в начальных классах / я очень ...,/ когда я была в начальных классах / я очень /...
PD-004-pic-1-adventure,,// мама с ребенком в коляске решила зайти в м...,/ мама с ребенком в коляске решила зайти в маг...


In [22]:
df = pd.merge(raw_transcript_lex_pt, df[['outer', 'discourse', 'altered']], on="File", how="outer")
df = df.fillna('')
# df['Transcript_new'] = df.apply(lambda a: a['Transcript_new'] if (('//' in a['Transcript_new'] and not '//' in ['Transcript_lex']) or len(a['Transcript_new']) > len(a['Transcript_lex'])) else a['Transcript_lex'], axis=1)
df = df.rename({'Transcript_lex':'pt'}, axis='columns')
df

,pt,outer,discourse,altered
File,,,,
N-01-instr-1-table,// необходимо на пол положить столешницу так ...,,,
N-01-pers-1-present,// самым запоминающимся подарком был последни...,,,
N-01-pic-1-sportsman,// человек был очень завистливый // он увиде...,,,
N-02-instr-1-chair,// переверните крышку табурета тыльной сторон...,,,
N-02-pers-1-present,// однажды / когда я еще училась в школе -/ к...,,,
...,...,...,...,...
PN-327-pic-1-sportsman,,/ жил один человек / который был очень худой и...,// жил один человек / который был очень худой...,// жил один человек / который был очень худой...
PD-004-pers-1-present,,// когда я была в начальных классах / я очень ...,/ когда я была в начальных классах / я очень /...,
PD-004-pic-1-adventure,,// мама с ребенком в коляске решила зайти в м...,/ мама с ребенком в коляске решила зайти в маг...,


In [23]:
added_ids = list(set(additional_transcript.index).difference(set(df.index)))
df = pd.merge(additional_transcript.loc[added_ids], df[['outer', 'discourse', 'altered', 'pt']], on="File", how="outer")
df = df.fillna('')
# df['Transcript_new'] = df.apply(lambda a: a['Transcript_new'] if (('//' in a['Transcript_new'] and not '//' in ['Transcript_lex']) or len(a['Transcript_new']) > len(a['Transcript_lex'])) else a['Transcript_lex'], axis=1)
df = df.rename({'Transcript':'add'}, axis='columns')
df

,add,outer,discourse,altered,pt
File,,,,,
PD-111-instr-1-chair,// ну как-то первый // я так понимаю // вот эт...,,,,
PD-130-instr-1-chair,// итак инструкция к сборке табуреток // что и...,,,,
PD-119-pic-1-sportsman,// идет здоровый мужик качок // хочет заставит...,,,,
PD-118-instr-1-chair,// так на первой картинке изображено /как соед...,,,,
PD-111-pic-1-adventure,// ребенок // главный герой ребенок // тут баб...,,,,
...,...,...,...,...,...
PN-327-pic-1-sportsman,,/ жил один человек / который был очень худой и...,// жил один человек / который был очень худой...,// жил один человек / который был очень худой...,
PD-004-pers-1-present,,// когда я была в начальных классах / я очень ...,/ когда я была в начальных классах / я очень /...,,
PD-004-pic-1-adventure,,// мама с ребенком в коляске решила зайти в м...,/ мама с ребенком в коляске решила зайти в маг...,,


In [24]:
def detect_best_annotation(series):
    # filter out empty strings
    candidates = [x for x in series.items() if x[1]]
    if len(candidates) == 1:
        # if only one is not none select it
        return candidates[0]
    else:
        # if several are not none
        utt_candidates = [x for x in candidates if '//' in x[1]]
        if len(utt_candidates) == 1:
            # if one has // and others do not select it
            return utt_candidates[0] 
        elif len(utt_candidates) == 0:
            # if none have // report
            return ('NO_UTT', '')
        else:
            # if several have // report
            res = sorted(utt_candidates, key=lambda x: x[1].count('\\'), reverse=False)[0]
            return res

In [25]:
df['src_res'] = df.apply(lambda x: detect_best_annotation(x), axis=1)
df['src'] = df['src_res'].apply(lambda x: x[0])
df['res'] = df['src_res'].apply(lambda x: x[1])
df

,add,outer,discourse,altered,pt,src_res,src,res
File,,,,,,,,
PD-111-instr-1-chair,// ну как-то первый // я так понимаю // вот эт...,,,,,"(add, // ну как-то первый // я так понимаю // ...",add,// ну как-то первый // я так понимаю // вот эт...
PD-130-instr-1-chair,// итак инструкция к сборке табуреток // что и...,,,,,"(add, // итак инструкция к сборке табуреток //...",add,// итак инструкция к сборке табуреток // что и...
PD-119-pic-1-sportsman,// идет здоровый мужик качок // хочет заставит...,,,,,"(add, // идет здоровый мужик качок // хочет за...",add,// идет здоровый мужик качок // хочет заставит...
PD-118-instr-1-chair,// так на первой картинке изображено /как соед...,,,,,"(add, // так на первой картинке изображено /ка...",add,// так на первой картинке изображено /как соед...
PD-111-pic-1-adventure,// ребенок // главный герой ребенок // тут баб...,,,,,"(add, // ребенок // главный герой ребенок // т...",add,// ребенок // главный герой ребенок // тут баб...
...,...,...,...,...,...,...,...,...
PN-327-pic-1-sportsman,,/ жил один человек / который был очень худой и...,// жил один человек / который был очень худой...,// жил один человек / который был очень худой...,,"(discourse, // жил один человек / который был...",discourse,// жил один человек / который был очень худой...
PD-004-pers-1-present,,// когда я была в начальных классах / я очень ...,/ когда я была в начальных классах / я очень /...,,,"(outer, // когда я была в начальных классах / ...",outer,// когда я была в начальных классах / я очень ...
PD-004-pic-1-adventure,,// мама с ребенком в коляске решила зайти в м...,/ мама с ребенком в коляске решила зайти в маг...,,,"(outer, // мама с ребенком в коляске решила з...",outer,// мама с ребенком в коляске решила зайти в м...


In [26]:
df[df['src'] == 'NO_UTT'] 

,add,outer,discourse,altered,pt,src_res,src,res
File,,,,,,,,


In [27]:
len(df[df['src'] == 'altered'])

15

In [28]:
len(df[df['src'] == 'add'])

67

In [29]:
df['utteranced'] = df['res'].apply(lambda x: '//' in x if type(x) == str else False)
df.head()

,add,outer,discourse,altered,pt,src_res,src,res,utteranced
File,,,,,,,,,
PD-111-instr-1-chair,// ну как-то первый // я так понимаю // вот эт...,,,,,"(add, // ну как-то первый // я так понимаю // ...",add,// ну как-то первый // я так понимаю // вот эт...,True
PD-130-instr-1-chair,// итак инструкция к сборке табуреток // что и...,,,,,"(add, // итак инструкция к сборке табуреток //...",add,// итак инструкция к сборке табуреток // что и...,True
PD-119-pic-1-sportsman,// идет здоровый мужик качок // хочет заставит...,,,,,"(add, // идет здоровый мужик качок // хочет за...",add,// идет здоровый мужик качок // хочет заставит...,True
PD-118-instr-1-chair,// так на первой картинке изображено /как соед...,,,,,"(add, // так на первой картинке изображено /ка...",add,// так на первой картинке изображено /как соед...,True
PD-111-pic-1-adventure,// ребенок // главный герой ребенок // тут баб...,,,,,"(add, // ребенок // главный герой ребенок // т...",add,// ребенок // главный герой ребенок // тут баб...,True


In [30]:
df[~df['utteranced']]

,add,outer,discourse,altered,pt,src_res,src,res,utteranced
File,,,,,,,,,


In [31]:
res_df = pd.DataFrame()
res_df['Transcript'] = df['res']

In [32]:
res_df['x'] = res_df.index.to_series().apply(lambda x: (i if i != 'present+ak' else 'present' for i in x.split('-')))
res_df[['group', 'id', 'task_type', 'date', 'task']] = pd.DataFrame(res_df.x.tolist(), index= res_df.index)
res_df['person_id_date'] = res_df['group'] + res_df['id'] + '_' + res_df['date']
res_df['person_id'] = res_df['group'] + res_df['id']
res_df = res_df.drop('x', axis=1)

In [33]:
res_df.head()

,Transcript,group,id,task_type,date,task,person_id_date,person_id
File,,,,,,,,
PD-111-instr-1-chair,// ну как-то первый // я так понимаю // вот эт...,PD,111,instr,1,chair,PD111_1,PD111
PD-130-instr-1-chair,// итак инструкция к сборке табуреток // что и...,PD,130,instr,1,chair,PD130_1,PD130
PD-119-pic-1-sportsman,// идет здоровый мужик качок // хочет заставит...,PD,119,pic,1,sportsman,PD119_1,PD119
PD-118-instr-1-chair,// так на первой картинке изображено /как соед...,PD,118,instr,1,chair,PD118_1,PD118
PD-111-pic-1-adventure,// ребенок // главный герой ребенок // тут баб...,PD,111,pic,1,adventure,PD111_1,PD111


In [34]:
res_df.date.value_counts()

1    428
Name: date, dtype: int64

In [35]:
len(res_df.person_id.unique())

145

In [36]:
res_df[['person_id', 'group']].reset_index().drop(columns=['File']).drop_duplicates().value_counts('group')

group
N     86
PD    38
PN    21
dtype: int64

In [37]:
pivot_df = pd.pivot_table(res_df, values='Transcript', index=['person_id_date'], columns=['task'],  aggfunc = 'sum')
pivot_df.to_csv('transcript_lex_by_task.tsv')

In [38]:
pivot_df.count()

task
adventure    66
bench        29
chair        56
party        35
present      62
sportsman    66
table        30
trip         42
winterday    41
dtype: int64

In [39]:
pivot_df

task,adventure,bench,chair,party,present,sportsman,table,trip,winterday
person_id_date,,,,,,,,,
N01_1,NaN,NaN,NaN,NaN,// самым запоминающимся подарком был последни...,// человек был очень завистливый // он увиде...,// необходимо на пол положить столешницу так ...,NaN,NaN
N02_1,NaN,NaN,// переверните крышку табурета тыльной сторон...,NaN,// однажды / когда я еще училась в школе -/ к...,// человек решил стать сильным / чтобы никто ...,NaN,NaN,NaN
N03_1,NaN,NaN,NaN,NaN,NaN,// толстый человек встретил очень худого // ...,// друзья нам с вами предстоит собрать стол /...,// ну пусть самое лучшее путешествие у меня б...,NaN
N04_1,NaN,NaN,NaN,// мой самый любимый и запоминающийся праздни...,NaN,// Вася Иванов всегда имел мечтал иметь спорт...,// инструкция очень интересная // берем // ...,NaN,NaN
N05_1,NaN,// итак // вам надо разделать доску на десят...,NaN,// ну самый запоминающийся праздник // ну да...,NaN,// идет как-то худощавый мужчина по улице по ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
PN229_1,NaN,NaN,NaN,// о самый запоминающийся праздник это мой по...,NaN,NaN,NaN,NaN,// один мальчик увидел в окно / что пошел сне...
PN231_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,// какая у меня скучная жизнь / подумала я ю ...,// маленький мальчик / который скорее всего т...
PN236_1,NaN,NaN,NaN,NaN,// я с детсва очень любила праздновать дни ро...,NaN,NaN,NaN,// однажды морозным зимним утром маленький ма...


### Add data from bachelor's

In [40]:
b_df = pd.read_csv(path + 'old_text_data_w_new_ids.csv', index_col=0)
b_df.columns=['adventure', 'present', 'sportsman', 'chair']
b_df = b_df.applymap(replace_func)

In [41]:
b_df['person_id_date'] = [x.replace('-', '') + '_1' for x in b_df.index]
b_df = b_df.set_index('person_id_date')

In [42]:
id_to_drop = list(set(b_df.index).intersection(set(pivot_df.index)))

In [43]:
pivot_df = pd.concat([pivot_df, b_df.drop(index=id_to_drop)])
pivot_df.columns = pivot_df.columns.set_names('task')

#### Incorrect merge?

In [44]:
res_df.loc[[x for x in res_df.index if x.startswith('S-14')]]

,Transcript,group,id,task_type,date,task,person_id_date,person_id
File,,,,,,,,


In [45]:
b_df.loc['PD114_1']

adventure     // ребенка оставляют на на улице перед домом ...
present                                                       
sportsman    // ну ладно мужчина разговаривает с мужчиной н...
chair                                                         
Name: PD114_1, dtype: object

### Make into "sentences"

In [46]:
def to_sents(utts: str) -> str:
    if pd.isnull(utts):
        return utts
    return '. '.join([s.replace('-/', '').replace('/', '').replace('  ', ' ').strip().capitalize() for s in utts.split('//') if s])

In [47]:
pivot_df.iloc[0][4]

'//  самым запоминающимся подарком был последний подарок //  это был приглашение на одну из лекций //  лекция довольно дорогая //  она была об английском языке //  это было что-то необычное новое / потому что никто мне никогда не дарил никаких приглашений //  это было очень //  это был очень интересный опыт / сходить на эту лекцию //  и поэтому мне очень запомнился этот подарок'

In [48]:
pivot_df.iloc[0]['present']

'//  самым запоминающимся подарком был последний подарок //  это был приглашение на одну из лекций //  лекция довольно дорогая //  она была об английском языке //  это было что-то необычное новое / потому что никто мне никогда не дарил никаких приглашений //  это было очень //  это был очень интересный опыт / сходить на эту лекцию //  и поэтому мне очень запомнился этот подарок'

In [49]:
to_sents(pivot_df.iloc[0]['present'])

'Самым запоминающимся подарком был последний подарок. Это был приглашение на одну из лекций. Лекция довольно дорогая. Она была об английском языке. Это было что-то необычное новое потому что никто мне никогда не дарил никаких приглашений. Это было очень. Это был очень интересный опыт сходить на эту лекцию. И поэтому мне очень запомнился этот подарок'

In [50]:
pivot_df.applymap(to_sents).to_csv('transcript_lex_by_task_with_dots.tsv', sep='\t')

### long format for piping through nlp

In [51]:
pivot_df.applymap(to_sents).values.tolist()

[[nan,
  nan,
  nan,
  nan,
  'Самым запоминающимся подарком был последний подарок. Это был приглашение на одну из лекций. Лекция довольно дорогая. Она была об английском языке. Это было что-то необычное новое потому что никто мне никогда не дарил никаких приглашений. Это было очень. Это был очень интересный опыт сходить на эту лекцию. И поэтому мне очень запомнился этот подарок',
  'Человек был очень завистливый. Он увидел другого мужчину который был широкоплечий мускулистый. И он захотел стать таким же. Он увидел афишу. И увидел статью в газете что чтобы стать мускулистым необходимо заниматься. Он пытался. Но у него к сожалению ничего не получалось поэтому он сдался. Выкинул все выкинул все оборудование в мусорку. И пошел к портному чтобы просто себя визуально сделать мускулистым. Вот. То есть он просто решил сделать вид что он мускулистый и сильный',
  'Необходимо на пол положить столешницу так чтобы черные точки смотрели вверх чтобы их было видно. Затем сверху нужно положить столеш

In [52]:
pivot_df.reset_index(inplace=True)

In [53]:
pivot_df

task,person_id_date,adventure,bench,chair,party,present,sportsman,table,trip,winterday
0,N01_1,NaN,NaN,NaN,NaN,// самым запоминающимся подарком был последни...,// человек был очень завистливый // он увиде...,// необходимо на пол положить столешницу так ...,NaN,NaN
1,N02_1,NaN,NaN,// переверните крышку табурета тыльной сторон...,NaN,// однажды / когда я еще училась в школе -/ к...,// человек решил стать сильным / чтобы никто ...,NaN,NaN,NaN
2,N03_1,NaN,NaN,NaN,NaN,NaN,// толстый человек встретил очень худого // ...,// друзья нам с вами предстоит собрать стол /...,// ну пусть самое лучшее путешествие у меня б...,NaN
3,N04_1,NaN,NaN,NaN,// мой самый любимый и запоминающийся праздни...,NaN,// Вася Иванов всегда имел мечтал иметь спорт...,// инструкция очень интересная // берем // ...,NaN,NaN
4,N05_1,NaN,// итак // вам надо разделать доску на десят...,NaN,// ну самый запоминающийся праздник // ну да...,NaN,// идет как-то худощавый мужчина по улице по ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
178,PD116_1,// здесь бабушка и говорит сыну / я имею ввиду...,NaN,// сначала вставляется одна деталь ножек / спр...,NaN,// мне подарили в этом году ветчерину для крас...,// мужчина комплексует по поводу своего тела ...,NaN,NaN,NaN
179,PD121_1,// ну началось с того / что малыш с мамой с ко...,NaN,,NaN,,// ну здесь встретились два мужчины // один оч...,NaN,NaN,NaN
180,PD123_1,// мама пошла с ребенком в магазин // и его ос...,NaN,,NaN,,// он похож на такого человека // злой злой че...,NaN,NaN,NaN
181,PD124_1,// женщина пошла в магазин // и чтобы // и пер...,NaN,,NaN,,// жил был начальник // и вернее рабочий этого...,NaN,NaN,NaN


In [54]:
long = pivot_df.melt(id_vars=['person_id_date'], value_name='transcript').sort_values(['person_id_date', 'task']).reset_index().drop(columns=['index'])

In [55]:
long.dropna()

,person_id_date,task,transcript
4,N01_1,present,// самым запоминающимся подарком был последни...
5,N01_1,sportsman,// человек был очень завистливый // он увиде...
6,N01_1,table,// необходимо на пол положить столешницу так ...
11,N02_1,chair,// переверните крышку табурета тыльной сторон...
13,N02_1,present,// однажды / когда я еще училась в школе -/ к...
...,...,...,...
1636,PN238_1,trip,// ну в в этом мае у меня был свободный от уч...
1637,PN238_1,winterday,// маленький мальчик увидел / что на улице ид...
1639,PN327_1,bench,// так // в общем у нас есть одна большая си...
1643,PN327_1,sportsman,// жил один человек / который был очень худой...


In [56]:
pd.pivot(long, columns='task', index='person_id_date')

transcript                                                     \
task            adventure                                              bench   
person_id_date                                                                 
N01_1                 NaN                                                NaN   
N02_1                 NaN                                                NaN   
N03_1                 NaN                                                NaN   
N04_1                 NaN                                                NaN   
N05_1                 NaN  //  итак //  вам надо разделать доску на десят...   
...                   ...                                                ...   
PN229_1               NaN                                                NaN   
PN231_1               NaN                                                NaN   
PN236_1               NaN                                                NaN   
PN238_1               NaN                                                NaN   
PN327_1               NaN  //  так //  в общем у нас есть одна большая си...   

                                                                   \
task                                                        chair   
person_id_date                                                      
N01_1                                                         NaN   
N02_1           //  переверните крышку табурета тыльной сторон...   
N03_1                                                         NaN   
N04_1                                                         NaN   
N05_1                                                         NaN   
...                                                           ...   
PN229_1                                                       NaN   
PN231_1                                                       NaN   
PN236_1                                                       NaN   
PN238_1                                                       NaN   
PN327_1                                                       NaN   

                                                                   \
task                                                        party   
person_id_date                                                      
N01_1                                                         NaN   
N02_1                                                         NaN   
N03_1                                                         NaN   
N04_1           //  мой самый любимый и запоминающийся праздни...   
N05_1           //  ну самый запоминающийся праздник //  ну да...   
...                                                           ...   
PN229_1         //  о самый запоминающийся праздник это мой по...   
PN231_1                                                       NaN   
PN236_1                                                       NaN   
PN238_1                                                       NaN   
PN327_1                                                       NaN   

                                                                   \
task                                                      present   
person_id_date                                                      
N01_1           //  самым запоминающимся подарком был последни...   
N02_1           //  однажды / когда я еще училась в школе -/ к...   
N03_1                                                         NaN   
N04_1                                                         NaN   
N05_1                                                         NaN   
...                                                           ...   
PN229_1                                                       NaN   
PN231_1                                                       NaN   
PN236_1         //  я с детсва очень любила праздновать дни ро...   
PN238_1                                                       NaN   
PN327_1                                                       N

### result processing

In [57]:
ex_1, ex_2 = [{'LM': {'m_lcoh': 0.33901671855345555, 'm_gcoh' : 0.5066645289197435, 'm_cgcoh': 0.5758008520071934, 'm_scoh': 0.240011725527549, 'm_sporb': 0.8207113922518843, 'm_bert_lcoh': 0.8989571, 'm_bert_gcoh': 0.94272983, 'm_bert_cgcoh': 0.947437, 'm_bert_scoh': 0.89258766}, 'syntactic': {'n_sents': 35, 'mean_sent_len': 5.257142857142857, 'std_sent_len': 2.4300835628531336, 'min_sent_len': 2, 'max_sent_len': 10, 'SPACE': 0.016304347826086956, 'NOUN': 0.13043478260869565, 'ADV': 0.24456521739130435, 'PUNCT': 0.20108695652173914, 'DET': 0.02717391304347826, 'VERB': 0.18478260869565216, 'PRON': 0.07065217391304347, 'CCONJ': 0.059782608695652176, 'AUX': 0.016304347826086956, 'PART': 0.005434782608695652, 'PROPN': 0.021739130434782608, 'ADJ': 0.016304347826086956, 'SCONJ': 0.005434782608695652}, 'lexical': {'n_words': 184, 'LTR': 0.532608695652174, 'MALTR': 0.8729885057471266}, 'graph': {'number_of_nodes': 63.392857142857146, 'number_of_edges': 99.0, 'PE': 7.130952380952381, 'LCC': 63.392857142857146, 'LSC': 61.04761904761905, 'degree_average': 3.1250790101673496, 'degree_std': 4.849276919361771, 'L1': 0.0, 'L2': 2.5, 'L3': 44.214285714285715}}, {'LM': {'m_lcoh': 0.35538894256345815, 'm_gcoh': 0.5945206141486934, 'm_cgcoh': 0.6295405924558081, 'm_scoh': 0.37093874984625225, 'm_sporb': 0.8744863314848197, 'm_bert_lcoh': 0.9021508, 'm_bert_gcoh': 0.9449449, 'm_bert_cgcoh': 0.9499106, 'm_bert_scoh': 0.89935684}, 'syntactic': {'n_sents': 39, 'mean_sent_len': 5.051282051282051, 'std_sent_len': 2.0121262889039433, 'min_sent_len': 2, 'max_sent_len': 10, 'SPACE': 0.02030456852791878, 'PROPN': 0.04060913705583756, 'NOUN': 0.17258883248730963, 'PUNCT': 0.20304568527918782, 'AUX': 0.025380710659898477, 'ADV': 0.19796954314720813, 'PRON': 0.07614213197969544, 'VERB': 0.15736040609137056, 'ADJ': 0.02030456852791878, 'CCONJ': 0.04568527918781726, 'SCONJ': 0.01015228426395939, 'PART': 0.015228426395939087, 'DET': 0.01015228426395939, 'NUM': 0.005076142131979695}, 'lexical': {'n_words': 197, 'LTR': 0.49746192893401014, 'MALTR': 0.8545454545454547}, 'graph': {'number_of_nodes': 56.01030927835052, 'number_of_edges': 99.0, 'PE': 10.54639175257732, 'LCC': 56.01030927835052, 'LSC': 54.144329896907216, 'degree_average': 3.5398917163643286, 'degree_std': 4.774675816630022, 'L1': 0.0, 'L2': 2.7010309278350517, 'L3': 54.34020618556701}}]

In [58]:
ex_1_1 = pd.DataFrame([pd.DataFrame.from_dict(ex_1, orient='index').stack()])
ex_2_1 = pd.DataFrame([pd.DataFrame.from_dict(ex_2, orient='index').stack()])

In [59]:
ex_2_1

LM                                                                  \
     m_lcoh    m_gcoh   m_cgcoh    m_scoh   m_sporb m_bert_lcoh m_bert_gcoh   
0  0.355389  0.594521  0.629541  0.370939  0.874486    0.902151    0.944945   

                           syntactic  ...           graph                  \
  m_bert_cgcoh m_bert_scoh   n_sents  ... number_of_nodes number_of_edges   
0     0.949911    0.899357      39.0  ...       56.010309            99.0   

                                                                            \
          PE        LCC       LSC degree_average degree_std   L1        L2   
0  10.546392  56.010309  54.14433       3.539892   4.774676  0.0  2.701031   

              
          L3  
0  54.340206  

[1 rows x 41 columns]

In [60]:
import numpy as np
empty_df = pd.DataFrame([np.nan])
empty_df.columns = pd.MultiIndex.from_tuples([('type', 'metric')])
empty_df

,type
,metric
0,NaN


In [61]:
ex_dict = dict((y, x) for x, y in zip([ex_1_1, ex_2_1, empty_df], ["chair", "child", 'empty']))

In [62]:
res = pd.concat(ex_dict.values(), keys=ex_dict.keys(), names=["TASK"], axis=1)
res

TASK     chair                                                      \
            LM                                                       
        m_lcoh    m_gcoh   m_cgcoh    m_scoh   m_sporb m_bert_lcoh   
0     0.339017  0.506665  0.575801  0.240012  0.820711    0.898957   

TASK                                                 ...           child  \
                                          syntactic  ...           graph   
     m_bert_gcoh m_bert_cgcoh m_bert_scoh   n_sents  ... number_of_edges   
0        0.94273     0.947437    0.892588      35.0  ...            99.0   

TASK                                                                           \
                                                                                
             PE        LCC       LSC degree_average degree_std   L1        L2   
0     10.546392  56.010309  54.14433       3.539892   4.774676  0.0  2.701031   

TASK             empty  
                  type  
             L3 metric  
0     54.340206    NaN  

[1 rows x 82 columns]

In [63]:
ex_res = pd.concat([res, res], axis=0)
ex_res

TASK     chair                                                      \
            LM                                                       
        m_lcoh    m_gcoh   m_cgcoh    m_scoh   m_sporb m_bert_lcoh   
0     0.339017  0.506665  0.575801  0.240012  0.820711    0.898957   
0     0.339017  0.506665  0.575801  0.240012  0.820711    0.898957   

TASK                                                 ...           child  \
                                          syntactic  ...           graph   
     m_bert_gcoh m_bert_cgcoh m_bert_scoh   n_sents  ... number_of_edges   
0        0.94273     0.947437    0.892588      35.0  ...            99.0   
0        0.94273     0.947437    0.892588      35.0  ...            99.0   

TASK                                                                           \
                                                                                
             PE        LCC       LSC degree_average degree_std   L1        L2   
0     10.546392  56.010309  54.14433       3.539892   4.774676  0.0  2.701031   
0     10.546392  56.010309  54.14433       3.539892   4.774676  0.0  2.701031   

TASK             empty  
                  type  
             L3 metric  
0     54.340206    NaN  
0     54.340206    NaN  

[2 rows x 82 columns]

In [64]:
ex_res['ids'] = [1, 2]
ex_res.set_index('ids')

TASK     chair                                                      \
            LM                                                       
        m_lcoh    m_gcoh   m_cgcoh    m_scoh   m_sporb m_bert_lcoh   
ids                                                                  
1     0.339017  0.506665  0.575801  0.240012  0.820711    0.898957   
2     0.339017  0.506665  0.575801  0.240012  0.820711    0.898957   

TASK                                                 ...           child  \
                                          syntactic  ...           graph   
     m_bert_gcoh m_bert_cgcoh m_bert_scoh   n_sents  ... number_of_edges   
ids                                                  ...                   
1        0.94273     0.947437    0.892588      35.0  ...            99.0   
2        0.94273     0.947437    0.892588      35.0  ...            99.0   

TASK                                                                           \
                                                                                
             PE        LCC       LSC degree_average degree_std   L1        L2   
ids                                                                             
1     10.546392  56.010309  54.14433       3.539892   4.774676  0.0  2.701031   
2     10.546392  56.010309  54.14433       3.539892   4.774676  0.0  2.701031   

TASK             empty  
                  type  
             L3 metric  
ids                     
1     54.340206    NaN  
2     54.340206    NaN  

[2 rows x 82 columns]

In [65]:
#     start = datetime.now()
#     config_de = Config(lang='de')
#     df = pd.read_csv('/Users/galina.ryazanskaya/Downloads/thesis?/code?/split_questions_cp_0.tsv', sep='\t',
#                      index_col=0).dropna()
#     df.reset_index(inplace=True)
#     long = df.melt(id_vars=['index'], value_name='transcript', var_name='task').sort_values(['index', 'task'])
#     texts = long['transcript'].to_list()
#     print('time for read and melt:', datetime.now() - start)

#     start = datetime.now()
#     vals = list(pipe_texts(texts, config_de))
#     print('time for pipe:', datetime.now() - start)
#     long['values'] = vals

#     start = datetime.now()
#     for_pd = []
#     for_entire_pd = []
#     cur_vals = ['', []]
#     for i, (ix, row) in enumerate(long.iterrows()):
#         if row['task'] == 'preprocessed_transcript':
#             dp = values_to_flat_dict(row['values'])
#             for_entire_pd.append(dp)
#         else:
#             if i > 0 and row['index'] != long.iloc[i-1]['index']:
#                 vals = average_values(cur_vals[-1])
#                 vals = values_to_flat_dict(vals)
#                 vals['index'] = cur_vals[0]
#                 for_pd.append(vals)
#                 cur_vals = ['', []]
#             else:
#                 cur_vals[0] = row['index']
#                 cur_vals[-1].append(row['values'])
#     new_df = pd.DataFrame(for_pd)
#     print('time for pd:', datetime.now() - start)

### view processed result

In [66]:
df = pd.read_csv('/Users/galina.ryazanskaya/Downloads/thesis?/code?/processed_values/ru_both.tsv', sep='\t', index_col=0, header=[0,1,2])
df.dropna(axis=1, how='all', inplace=True)
df = df.rename(columns={'averaged': ' averaged'})
df.sort_index(axis=1, level=[0, 1], inplace=True)
df = df.rename(columns={' averaged': 'averaged'})
df

TASK              adventure                                                \
                         LM                                                 
               m_bert_cgcoh m_bert_gcoh m_bert_lcoh m_bert_scoh   m_cgcoh   
person_id_date                                                              
N01_1                   NaN         NaN         NaN         NaN       NaN   
N02_1                   NaN         NaN         NaN         NaN       NaN   
N03_1                   NaN         NaN         NaN         NaN       NaN   
N04_1                   NaN         NaN         NaN         NaN       NaN   
N05_1                   NaN         NaN         NaN         NaN       NaN   
...                     ...         ...         ...         ...       ...   
PD125_1            0.946462    0.938903    0.874884    0.847150  0.886114   
PD126_1            0.965636    0.945550    0.931653    0.927121  0.784499   
PD127_1            0.954185    0.944557    0.946435    0.812391  0.999997   
PD129_1            0.913741    0.906275    0.883311    0.861123  0.939872   
PD130_1            0.960530    0.950267    0.897895    0.886008  0.878541   

TASK                                                          ... winterday  \
                                                       graph  ... syntactic   
                  m_gcoh    m_lcoh    m_scoh   m_sporb    L1  ...      PRON   
person_id_date                                                ...             
N01_1                NaN       NaN       NaN       NaN   NaN  ...       NaN   
N02_1                NaN       NaN       NaN       NaN   NaN  ...       NaN   
N03_1                NaN       NaN       NaN       NaN   NaN  ...       NaN   
N04_1                NaN       NaN       NaN       NaN   NaN  ...       NaN   
N05_1                NaN       NaN       NaN       NaN   NaN  ...       NaN   
...                  ...       ...       ...       ...   ...  ...       ...   
PD125_1         0.878837  0.736865  0.758299  0.985037   0.0  ...       NaN   
PD126_1         0.747864  0.555280  0.483353  0.697083   0.0  ...       NaN   
PD127_1         0.999994  0.999987  0.999985  0.668529   0.0  ...       NaN   
PD129_1         0.928992  0.846044  0.827163  0.995910   0.0  ...       NaN   
PD130_1         0.868662  0.757499  0.747786  0.912958   0.0  ...       NaN   

TASK                                                                           \
                                                                                
               PROPN PUNCT SCONJ VERB max_sent_len mean_sent_len min_sent_len   
person_id_date                                                                  
N01_1            NaN   NaN   NaN  NaN          NaN           NaN          NaN   
N02_1            NaN   NaN   NaN  NaN          NaN           NaN          NaN   
N03_1            NaN   NaN   NaN  NaN          NaN           NaN          NaN   
N04_1            NaN   NaN   NaN  NaN          NaN           NaN          NaN   
N05_1            NaN   NaN   NaN  NaN          NaN           NaN          NaN   
...              ...   ...   ...  ...          ...           ...          ...   
PD125_1          NaN   NaN   NaN  NaN          NaN           NaN          NaN   
PD126_1          NaN   NaN   NaN  NaN          NaN           NaN          NaN   
PD127_1          NaN   NaN   NaN  NaN          NaN           NaN          NaN   
PD129_1          NaN   NaN   NaN  NaN          NaN           NaN          NaN   
PD130_1          NaN   NaN   NaN  NaN          NaN           NaN          NaN   

TASK                                 
                                     
               n_sents std_sent_len  
person_id_date                       
N01_1              NaN          NaN  
N02_1              NaN          NaN  
N03_1              NaN          NaN  
N04_1              NaN          NaN  
N05_1              NaN          NaN  
...                ...          ...  
PD125_1            NaN          NaN  
PD126_1            NaN  

In [67]:
df.to_csv('/Users/galina.ryazanskaya/Downloads/thesis?/code?/rus_postp.tsv', sep='\t')

In [68]:
df.sort_index(axis=1, level=[1, 2], inplace=True)
df

TASK,adventure,bench,chair,party,present,sportsman,table,trip,winterday,adventure,...,winterday,adventure,bench,chair,party,present,sportsman,table,trip,winterday
,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM,...,syntactic,syntactic,syntactic,syntactic,syntactic,syntactic,syntactic,syntactic,syntactic,syntactic
,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_cgcoh,m_bert_gcoh,...,n_sents,std_sent_len,std_sent_len,std_sent_len,std_sent_len,std_sent_len,std_sent_len,std_sent_len,std_sent_len,std_sent_len
person_id_date,,,,,,,,,,,,,,,,,,,,,
N01_1,NaN,NaN,NaN,NaN,0.972446,0.910135,0.967122,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.919104,3.620420,4.582091,NaN,NaN
N02_1,NaN,NaN,0.907379,NaN,0.979091,0.979785,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.256108,NaN,2.752524,2.424158,NaN,NaN,NaN
N03_1,NaN,NaN,NaN,NaN,NaN,0.958732,0.963964,0.965199,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.256083,4.908583,5.425634,NaN
N04_1,NaN,NaN,NaN,0.979656,NaN,0.939096,0.963407,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.559026,NaN,5.627919,7.791666,NaN,NaN
N05_1,NaN,0.958242,NaN,0.966964,NaN,0.924580,NaN,NaN,NaN,NaN,...,NaN,NaN,7.203496,NaN,4.151091,NaN,5.838799,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PD125_1,0.946462,NaN,NaN,NaN,NaN,0.942031,NaN,NaN,NaN,0.938903,...,NaN,2.683620,NaN,NaN,NaN,NaN,3.130495,NaN,NaN,NaN
